# Introduction

On part donc sur Option “Intent TFLite + candidate-driven” :

Entraînement Python d’un petit classifieur d’intentions (char-level) → export TFLite.

Sur Android : on recharge le modèle TFLite pour prédire l’intent, puis on récupère la valeur par candidate-driven (pas de regex, pas de NER — simple et robuste).

Ci-dessous, un notebook découpé par cellules : données → entraînement → export → fichiers à embarquer, + les notes Android pour reproduire la vectorisation et brancher le candidate-driven.

Ce qu'on veut faire:

- intent_classifier.tflite (modèle compact, quantized).

- char_vocab.txt (vocabulaire caractères).

- labels.txt (ordre des intents).

- Un pipeline Android très léger :

    - vectoriser la phrase (caractères → indices via char_vocab.txt),

    - Interpreter.run() → intent,

    - candidate-driven pour matcher la valeur du champ prédit (cosine n-grammes),

    - comparaison à la fiche JSON → OK / INCERTAIN / KO.

In [1]:
import os, json, random, unicodedata, re
import numpy as np
from typing import List, Dict

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2025-11-05 16:15:50.509316: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

# Intents (ordre fixe = indices softmax)
INTENTS = [
    "PATIENT_IDENTITE",
    "PATIENT_NAISSANCE",
    "INTERVENTION_TYPE",
    "SITE_OPERATOIRE",
    "HEURE_PREVUE",
    "CHIRURGIEN",
    "ANESTHESISTE",
    "OTHER"
]
label2id = {l:i for i,l in enumerate(INTENTS)}
id2label = {i:l for l,i in label2id.items()}

# Longueur max (en caractères) pour l'entrée du modèle
SEQLEN = 200
# Vocab max (caractères les + fréquents)
VOCAB_SIZE = 300  # suffisant en char-level FR (tu peux monter à 500)
EMB_DIM = 64
HID_DIM = 64
EPOCHS = 10
BATCH = 64

# Normalisation "Android-like": lowercase, strip accents, espaces uniques

In [3]:
def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def normalize_text(s: str) -> str:
    s = s.lower().strip()
    s = strip_accents(s)
    s = re.sub(r'[^a-z0-9:/ \-]', ' ', s)  # garde chiffres, :, /, -, espace
    s = re.sub(r'\s+', ' ', s).strip()
    return s


# Dataset minimal (remplace X_raw/y_raw par tes données)

In [9]:
import pandas as pd
data = pd.read_csv("../data/intent_corpus.tsv", sep="\t", header=None, names=["text", "intent"], dtype=str)
data = data.dropna().sample(frac=1.0, random_state=SEED).reset_index(drop=True)
data.head()

,text,intent
0,type d intervention cholecystectomie,INTERVENTION_TYPE
1,née le 1956-09-23,PATIENT_NAISSANCE
2,silence en salle,OTHER
3,on dit patient Alexandre Marchand,PATIENT_IDENTITE
4,site opératoire genou à droite,SITE_OPERATOIRE


In [10]:
data['text'] = data['text'].apply(normalize_text)
data = data[data['intent'].isin(INTENTS)].reset_index(drop=True)
data['intent_id'] = data['intent'].apply(lambda x: label2id[x])
X = data['text'].tolist()
y = data['intent_id'].tolist()

# Vectoriseur caractères

In [11]:
# On pré-normalise (ci-dessus), donc standardize=None. On découpe en caractères.
vectorizer = layers.TextVectorization(
    standardize=None,
    split="character",
    output_mode="int",
    output_sequence_length=SEQLEN,
    max_tokens=VOCAB_SIZE
)

# Le vectorizer doit voir des textes pour construire le vocab
vectorizer.adapt(np.array(X))

# Sauvegarde du vocab (indexation identique pour Android)
vocab = vectorizer.get_vocabulary()  # vocab[0] = '' (padding), vocab[1] = '[UNK]'
with open("char_vocab.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(vocab))
print("Vocab size:", len(vocab))

# Prépare les tenseurs d'entrée/sortie
X_ids = vectorizer(np.array(X)).numpy()
Y_1h = tf.keras.utils.to_categorical(y, num_classes=len(INTENTS))

Vocab size: 42


## Enregister vocab dans un fichier txt

In [22]:
with open("vocab.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(vocab))
print(f"vocab.txt écrit ({len(vocab)} tokens)")

vocab.txt écrit (42 tokens)


In [25]:
# Ensure the vectorizer is adapted using the existing X (texts) variable
vectorizer.adapt(np.array(X))
vocab = vectorizer.get_vocabulary()
with open("char_vocab.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(vocab))


# Split + modèle

In [12]:
from sklearn.model_selection import train_test_split
Xtr, Xva, Ytr, Yva = train_test_split(X_ids, Y_1h, test_size=0.2, random_state=SEED, stratify=y)

inputs = keras.Input(shape=(SEQLEN,), dtype="int32", name="char_ids")
x = layers.Embedding(input_dim=len(vocab), output_dim=EMB_DIM, mask_zero=True)(inputs)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(HID_DIM, activation="relu")(x)
outputs = layers.Dense(len(INTENTS), activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ char_ids            │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 200, 64)   │      2,688 │ char_ids[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 200)       │          0 │ char_ids[0][0]    │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ embedding[0][0],  │
│ (GlobalAveragePool… │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 8)         │        520 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,368 (28.78 KB)

 Trainable params: 7,368 (28.78 KB)

 Non-trainable params: 0 (0.00 B)

# Train & eval

In [13]:
history = model.fit(
    Xtr, Ytr,
    validation_data=(Xva, Yva),
    epochs=EPOCHS,
    batch_size=BATCH,
    verbose=1
)

# Petit test
def predict_intent_text(txt: str):
    t = normalize_text(txt)
    ids = vectorizer(np.array([t]))
    prob = model.predict(ids, verbose=0)[0]
    k = int(np.argmax(prob))
    return INTENTS[k], float(prob[k])

for t in ["le patient est claire martin", "site fid", "a 10h30", "bonjour"]:
    print(t, "->", predict_intent_text(t))


Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.4061 - loss: 1.8083 - val_accuracy: 0.6035 - val_loss: 1.4155
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.7587 - loss: 1.1040 - val_accuracy: 0.8355 - val_loss: 0.8231
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ -1s -20427us/step - accuracy: 0.8535 - loss: 0.6381 - val_accuracy: 0.8615 - val_loss: 0.5114
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8916 - loss: 0.4180 - val_accuracy: 0.9030 - val_loss: 0.3768
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9188 - loss: 0.3180 - val_accuracy: 0.9359 - val_loss: 0.3076
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9340 - loss: 0.2632 - val_accuracy: 0.9446 - val_loss: 0.2653
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9418 - loss: 0.2280 - val_accuracy: 0.9463 - val_loss: 0.2366
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9476 - loss: 0.2029 - val_accuracy: 0.942

# Export SavedModel -> TFLite (quantization dynamique)

In [17]:
# Pas besoin d’appeler model.save() avant
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # quantization dynamique (facultatif)
tflite_model = converter.convert()

with open("intent_classifier.tflite", "wb") as f:
    f.write(tflite_model)

# (labels/vocab restent inchangés)


INFO:tensorflow:Assets written to: /tmp/tmpzj8m3p0u/assets


INFO:tensorflow:Assets written to: /tmp/tmpzj8m3p0u/assets


Saved artifact at '/tmp/tmpzj8m3p0u'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200), dtype=tf.int32, name='char_ids')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  139804392095376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139804392091536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139804392093648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139804376531216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139804376531984: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1762357013.028340   28249 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1762357013.029199   28249 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-05 16:36:53.035216: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpzj8m3p0u
2025-11-05 16:36:53.036582: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-11-05 16:36:53.036665: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpzj8m3p0u
I0000 00:00:1762357013.052851   28249 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled
2025-11-05 16:36:53.054467: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-11-05 16:36:53.111583: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpzj8m3p0u
2025-11-05 16:36:53.128807: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

# INFERENCE

In [29]:
import numpy as np, unicodedata, re, tensorflow as tf

SEQLEN = 200  # même valeur que lors de l'entraînement

def strip_accents(s): 
    import unicodedata
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c)!='Mn')

def normalize_text(s: str) -> str:
    s = s.lower().strip()
    s = strip_accents(s)
    s = re.sub(r'[^a-z0-9:/ \-]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# Charger vocab et labels exportés
with open("char_vocab.txt", "r", encoding="utf-8") as f:
    vocab = [line.rstrip("\n") for line in f]
tok2id = {tok:i for i,tok in enumerate(vocab)}  # 0="" padding, 1="[UNK]"

with open("intents.txt", "r", encoding="utf-8") as f:
    LABELS = [line.rstrip("\n") for line in f]

def vectorize_chars(text: str, seqlen=SEQLEN):
    t = normalize_text(text)
    chars = list(t)  # split caractère
    ids = np.zeros((1, seqlen), dtype=np.int32)
    for i, ch in enumerate(chars[:seqlen]):
        ids[0, i] = tok2id.get(ch, tok2id.get("[UNK]", 1))
    return ids

# Charger le modèle TFLite
interpreter = tf.lite.Interpreter(model_path="intent_classifier.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def predict_intent_tflite(text: str):
    x = vectorize_chars(text)
    interpreter.set_tensor(input_details[0]['index'], x)  # int32 [1, SEQLEN]
    interpreter.invoke()
    probs = interpreter.get_tensor(output_details[0]['index'])[0]  # float32 [num_labels]
    k = int(np.argmax(probs))
    return LABELS[k], float(probs[k])

print(predict_intent_tflite("le patient est Paul Durant"))
print(predict_intent_tflite("site thoracique"))


('"PATIENT_IDENTITE",', 0.9954239726066589)
('"SITE_OPERATOIRE",', 0.9898533225059509)
